## Dataset
This <a href="https://www.kaggle.com/andrewmvd/face-mask-detection">dataset</a> contains 853 images belonging to the 3 classes, as well as their bounding boxes in the PASCAL VOC format.
The classes are:

- With mask
- Without mask
- Mask worn incorrectly

<b>Directory structure</b>

            Face Mask Detection
            │    
            ├───data
            │   ├───annotations
            │   │      .xml
            │   └───images
            │          .png
            ...

### Importing Libraries

In [1]:
import os
import cv2
import pickle
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.utils import to_categorical

C:\Users\LENOVO\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\LENOVO\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\LENOVO\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\LENOVO\AppData\Roaming\Python\Python37\s

### Reading image Path, Label, Bounding boxes from XML file

In [2]:
BASE_DIR = os.getcwd()
image_dir = os.path.join(BASE_DIR, 'data', 'images')
annot_dir = os.path.join(BASE_DIR, 'data', 'annotations')

In [3]:
label2category = {'without_mask': 0, 'with_mask': 1, 'mask_weared_incorrect': 2}
category2label = {v: k for k, v in label2category.items()}
datas = []

for root, dirs, files in os.walk(annot_dir):
    for file in files:
        tree = ET.parse(os.path.join(root, file))
        data = {'path': None, 'objects': []}
        data['path'] = os.path.join(image_dir, tree.find('filename').text)
        for obj in tree.findall('object'):
            label = label2category[obj.find('name').text]
            # top left co-ordinates
            xmin = int(obj.find('bndbox/xmin').text)
            ymin = int(obj.find('bndbox/ymin').text)
            # bottom right co-ordinates
            xmax = int(obj.find('bndbox/xmax').text)
            ymax = int(obj.find('bndbox/ymax').text)
            data['objects'].append([label, xmin, ymin, xmax, ymax])
        datas.append(data)

print('Total images :', len(datas))

Total images : 0


### What data looks like

In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

if len(datas) == 0:
    print("Error: No data available.")
else:
    index = np.random.randint(0, len(datas))
    if index >= len(datas):
        print("Error: Invalid index generated.")
    else:
        img = cv2.imread(datas[index]['path'])
        if img is None:
            print("Error: Failed to load image.")
        else:
            for (category, xmin, ymin, xmax, ymax) in datas[index]['objects']:
                # Draw bounding boxes
                cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1)
                cv2.putText(img, str(category), (xmin+2, ymin-3), cv2.FONT_HERSHEY_SIMPLEX, 0.38, (255,255,255), 2)

            # Show image
            plt.figure(figsize=(10, 6))
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.show()


Error: No data available.


### Get data and target 
Take <b>Region of interest</b> as data(X). 

And target as <b>categorical data</b>. i.e.

    0 -> [1, 0, 0]
    1 -> [0, 1, 0]
    2 -> [0, 0, 1]

In [ ]:
img_size = (100, 100)
X = []
Y = []

for data in datas:
    img = cv2.imread(data['path'])
    for (category, xmin, ymin, xmax, ymax) in data['objects']:
        roi = img[ymin : ymax, xmin : xmax]
        roi = cv2.resize(roi, (100, 100))
        data = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        target = to_categorical(category, num_classes=len(category2label))
        X.append(data)
        Y.append(target)
        
X = np.array(X)
Y = np.array(Y)

In [ ]:
np.save('data/X', X)
np.save('data/Y', Y)

In [ ]:
with open('data/category2label.pkl', 'wb') as pf:
    pickle.dump(category2label, pf)